Exploratory Data Analysis

In [72]:
import pandas as pd
import plotly.express as px

df = pd.read_parquet("../data/raw/SPY_1min_Jan2023.parquet")

In [73]:
print(df.head())
print(df.describe())

df = df.reset_index()

                                    open    high     low   close   volume  \
symbol timestamp                                                            
SPY    2023-01-03 09:00:00+00:00  384.00  386.00  384.00  385.84  14020.0   
       2023-01-03 09:01:00+00:00  385.84  386.14  385.79  386.00   4598.0   
       2023-01-03 09:02:00+00:00  386.04  386.23  386.04  386.10   2063.0   
       2023-01-03 09:03:00+00:00  386.13  386.45  386.12  386.45   3067.0   
       2023-01-03 09:04:00+00:00  386.45  386.54  386.45  386.51   2450.0   

                                  trade_count        vwap  
symbol timestamp                                           
SPY    2023-01-03 09:00:00+00:00        154.0  385.795007  
       2023-01-03 09:01:00+00:00         56.0  385.987750  
       2023-01-03 09:02:00+00:00         24.0  386.128000  
       2023-01-03 09:03:00+00:00         28.0  386.236025  
       2023-01-03 09:04:00+00:00         27.0  386.497272  
               open          high       

In [78]:
df['timestamp'] = pd.to_datetime(df['timestamp'])
df = df.set_index("timestamp")
df = df.between_time("09:30", "16:00")

df = df.reset_index()
df['trading_minute'] = range(len(df))

fig = px.line(
    df,
    x="trading_minute",
    y=["close", "vwap"],
    title="SPY Close vs VWAP Jan 2023 (1-min, Trading Hours Only)"
)

# Choose some points to label (e.g. every day’s open)
day_starts = df.groupby(df['timestamp'].dt.date)['trading_minute'].min()
tickvals = day_starts.values
ticktext = [str(d) for d in day_starts.index]

fig.update_xaxes(tickvals=tickvals, ticktext=ticktext)

fig.show()
